In [0]:
# Get resources dataset from Jiatai's Git
! git clone https://github.com/hjt486/DeepColorizer

# Get and confirm path
#! pwd

Cloning into 'DeepColorizer'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 9979 (delta 10), reused 92 (delta 4), pack-reused 9880
Receiving objects: 100% (9979/9979), 630.47 MiB | 32.49 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Checking out files: 100% (10589/10589), done.


In [0]:
# Force to install Keras 2.1.5
# (To resolve issue of saving\loading .h5 model)
!pip install keras==2.1.5
# Tensorflow
import tensorflow as tf
# Keras
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import TensorBoard
# Image pre/post processing
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
# Misc
import math
import numpy as np
import os
import random
# Display result in CoLab
from IPython.display import Image, display

# Define resource root path
path = '/content/DeepColorizer/datasets/selected/'

    100% |████████████████████████████████| 337kB 28.8MB/s 
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


Using TensorFlow backend.


In [0]:
# Load images
# Mixed contains 300 images, 200 landscapes, 100 portraits
X = []
file_limit = os.listdir(path + 'Train/Mixed/')[:100]
for filename in file_limit:
    X.append(img_to_array(load_img(path + 'Train/Mixed/' + filename)))
X = np.array(X, dtype=float)

# Split loaded images into train and test data set with 95:5
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain
Xval = X[split:]
Xval = 1.0/255*Xval

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Training parameters
# total = how many traing images
total = 300
batch_size = 100
epochs = 3250
# steps is automatically decided based on total and batch_size
steps = math.ceil(total/batch_size)

# Generate training data
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)
        
# Generate validation data
def image_a_b_gen2(batch_size):
    for batch in datagen.flow(Xval, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

In [0]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse', metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Train model      
model.fit_generator(image_a_b_gen(batch_size),
                    validation_data=image_a_b_gen2(batch_size),
                    epochs=epochs, steps_per_epoch=steps, validation_steps=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3256
3/3 [==============================] - 29s 10s/step - loss: 0.6956 - acc: 0.5057 - val_loss: 1.1945 - val_acc: 0.3653
Epoch 2/3256
3/3 [==============================] - 4s 1s/step - loss: 1.0032 - acc: 0.4998 - val_loss: 0.0443 - val_acc: 0.3542
Epoch 3/3256
3/3 [==============================] - 12s 4s/step - loss: 0.0212 - acc: 0.5041 - val_loss: 0.0967 - val_acc: 0.3566
Epoch 4/3256
3/3 [==============================] - 13s 4s/step - loss: 0.0262 - acc: 0.5042 - val_loss: 0.0270 - val_acc: 0.3674
Epoch 5/3256
3/3 [==============================] - 13s 4s/step - loss: 0.0120 - acc: 0.5048 - val_loss: 0.0273 - val_acc: 0.3797
Epoch 6/3256
3/3 [==============================] - 12s 4s/step - loss: 0.0121 - acc: 0.5112 - val_loss: 0.0265 - val_acc: 0.4073
Epoch 7/3256
3/3 [==============================] - 12s 4s/step - loss: 0.0121 - acc: 0.5191 - val_loss: 0.0281 - val_acc: 0.3365
Epoch 8/3256
3/3 [========================

In [0]:
# Save Model
model.save('mixed.h5')
print('Model saved 300x3x3250')
! ls
# Download model
# files.download('mixed.h5') 


Model saved 300x3x3250
DeepColorizer  mixed.h5  sample_data


In [0]:
# Load test images
image_path = path + 'Validate/1dZTRG.jpg'
# Process test image to grey scale
image = img_to_array(load_img(image_path))
color_me = np.array(image, dtype=float)
color_me = rgb2lab(1.0/255*image)[:,:,0]
color_me = color_me.reshape(1, 256, 256, 1)

# Apply model to generate A B channels
output = model.predict(color_me)
output = output * 128

# Merge LAB space and convert to RGB, save
cur = np.zeros((256, 256, 3))
cur[:,:,0] = color_me[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))
Image('img_result.png')